In [4]:
import sqlite3
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

def get_db(db_name):
	db = sqlite3.connect(db_name)
	return db

def close_db(db):
    db.close()

def load_article_id_map(db_name):
    db = get_db(db_name).cursor()
    db.execute(f"""SELECT title, date, origin_link, origin_id
                    FROM article_id_map 
                    WHERE date LIKE '2024%';""")
    rows = db.fetchall()
    return rows

# 테이블 생성 (이미 존재하면 생략)
def create(db_name):
    db = get_db(db_name)
    cursor = db.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS article (
            id INTEGER PRIMARY KEY,
            title TEXT NOT NULL,
            date INTEGER NOT NULL,
            content TEXT NOT NULL,
            link TEXT NOT NULL
        )
    ''')
    db.commit()  # 변경사항 저장
    close_db(db) # 접속 종료

def crawling(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # 요청에 실패하면 예외 발생
        soup = BeautifulSoup(response.text, 'html.parser')
        article_section = soup.find('section', class_='news_view')
        
        if article_section:
            # 광고 등 불필요한 태그를 제거
            for ad in article_section.find_all(['div', 'script', 'figure']):
                ad.decompose()
            
            # 남은 텍스트 추출
            article_text = article_section.get_text(separator=' ', strip=True)
            return article_text
        else:
            return None
    except requests.RequestException as e:
        print(f"HTTP 요청 실패: {e}")
        return None
    
def insert(db_name, rows):
    db = get_db(db_name)
    cursor = db.cursor()
    for row in tqdm(rows):
        content = crawling(row[2])
        if content:
            cursor.execute('''
                INSERT INTO article (id, title, date, content, link)
                VALUES (?, ?, ?, ?, ?)
            ''', (row[3], row[0], row[1], content, row[2])) # origin_id, title, date, content, link
    db.commit()
    close_db(db)

def select(db_name):
    db = get_db(db_name).cursor()
    db.execute(f"""SELECT id, title, date, content, link
                    FROM article;""")
    rows = db.fetchall()
    return rows

In [5]:
rows = load_article_id_map('dataset/pr-db.db')

In [6]:
len(rows)

32544

In [ ]:
# 새로운 DB 만들고 insert
create('dataset/2024.db')
insert('dataset/2024.db', rows)

In [7]:
rows = select('dataset/2024.db')
len(rows)

32294

In [9]:
import pandas as pd

df = pd.DataFrame(rows, columns=['id', 'title', 'date', 'content', 'link'])
df.head()

,id,title,date,content,link
0,122846231,中 군 고위간부 9명 전인대 대표 박탈… ‘로켓군’ 납품 관련 부패사건 연루 의혹,20240101,“리상푸 前국방부장 낙마와 연관” 리부장 후임엔 해군출신 둥쥔 중국 당국이 ‘반(反...,https://www.donga.com/news/Inter/article/all/2...
1,122846362,서울 12cm 기습 폭설… “이런 눈 또 온다”,20240101,13년만에 최대… 도로 곳곳 사고 방파제 추락-조난사고도 발생 “엘니뇨-온난화로 폭...,https://www.donga.com/news/Society/article/all...
2,122846400,"‘자산 301조원’ 머스크, 세계 최고 부자에",20240101,1년새 124조원 늘어… 증가액도 1위 아르노 LVMH 회장 2위로 밀려나 베이조스...,https://www.donga.com/news/People/article/all/...
3,122846403,나이지리아서 피랍 17일만에… 대우건설 韓직원 2명 풀려나,20240101,나이지리아에서 무장단체에 납치됐던 대우건설 한국인 직원 2명이 17일 만에 무사히 ...,https://www.donga.com/news/Economy/article/all...
4,122846406,"부모 학대로 다리 잃은 英 9세 소년, 최연소 훈장 받는다",20240101,혼자 의족-목발로 10km 걷기 도전 자기 이름 딴 재단 세워 32억원 모금 어린이...,https://www.donga.com/news/People/article/all/...


In [10]:
df.to_csv("dataset/2024.csv", index=False)